In [ ]:

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score


DATA_PATH = "spending_labeled_clusters.csv"
df = pd.read_csv(DATA_PATH)


FEATURES = ["Annual Income ($)", "Spending Score (1-100)"]
X = df[FEATURES].copy()

for col in FEATURES:
    if X[col].isna().any():
        X[col] = X[col].fillna(X[col].median())


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


print(" ELBOW CHECK (SSE)")
for k in range(1, 11):
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    model.fit(X_scaled)
    print(f"k={k} → SSE={model.inertia_:.2f}")


K = 3  # Choose based on SSE trend
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)

df["Cluster"] = labels.astype(int)

sil = silhouette_score(X_scaled, labels)
dbi = davies_bouldin_score(X_scaled, labels)

print("\n CLUSTER METRICS ")
print(f"Silhouette Score : {sil:.3f}")
print(f"Davies–Bouldin   : {dbi:.3f}")


centers_scaled = kmeans.cluster_centers_
centers_original = scaler.inverse_transform(centers_scaled)

centers_df = pd.DataFrame(centers_original, columns=FEATURES)
centers_df.index.name = "Cluster"

print("\n CLUSTER CENTERS (Original Units) ")
print(centers_df.round(2))


print("\nSANITY CHECK (3 SAMPLE ROWS) ")
sample_rows = df.sample(3, random_state=42)
print(sample_rows[FEATURES + ["Cluster"]])


OUT_PATH = "spending_labeled_clusters.csv"
df.to_csv(OUT_PATH, index=False)

print(f"\nSaved file: {OUT_PATH}")


 CLUSTER METRICS 
Silhouette Score : 0.381
Davies–Bouldin   : 0.862

 CLUSTER CENTERS (Original Units) 
         Annual Income ($)  Spending Score (1-100)
Cluster                                           
0                107951.13                   22.84
1                 41351.42                   46.55
2                107274.34                   77.92

SANITY CHECK (3 SAMPLE ROWS) 
     Annual Income ($)  Spending Score (1-100)  Cluster
521             149092                      69        2
737              19703                      84        1
740              58585                       2        0

Saved file: spending_labeled_clusters.csv
